In [82]:
from mtspHeuristic import *
import random

In [83]:
def generate_population(number_of_individuals, number_of_genes):
    population = []
    for _ in range(number_of_individuals):
        population.append([round(random.random(),3) for __ in range(number_of_genes)])
    return population
'''
a = generate_population(40, 5)
a2 = a[2].copy()
a[:5], fitness_function(a[0], cities_distances)
'''

'\na = generate_population(40, 5)\na2 = a[2].copy()\na[:5], fitness_function(a[0], cities_distances)\n'

In [84]:
def decoder_sequential_insertion(individual, number_of_salesmen, distance_matrix):
    
    indexed_individual = [(individual[item], item+2) for item in range(len(individual))] # Linear
    indexed_individual.sort(key = lambda x: x[0]) # "LogLinear"
    ordered_indexes = [item[1] for item in indexed_individual] # Linear
    initial_cities = ordered_indexes[:number_of_salesmen]
    other_cities = ordered_indexes[number_of_salesmen:]

    initial_solutions = [[1,i,1] for i in initial_cities]
    route_to_increment_index = 0
    final_city = 0
    city_index_to_insert = 0
    max_distance = float("inf")
    for new_city in other_cities:
        for route_index in range(0,len(initial_solutions)): # Para cada rota
            route = initial_solutions[route_index]
            for city_index in range(1,len(route)): # Para cada cidade já inserida na dita rota
                city_in_route = route[city_index]
                city_before_in_route = route[city_index-1]
                insertion_cost = distance_matrix[city_in_route-1][new_city-1] + distance_matrix[new_city-1][city_before_in_route-1] - distance_matrix[city_in_route-1][city_before_in_route-1]
                if insertion_cost < max_distance:
                    max_distance = insertion_cost
                    route_to_increment_index = route_index
                    city_index_to_insert = city_index
                    final_city = new_city
        city_to_insert = final_city
        initial_solutions[route_to_increment_index] = insert_in_vector(initial_solutions[route_to_increment_index], city_to_insert, city_index_to_insert)
        # Reset das variáveis do problema
        route_to_increment_index = 0
        max_distance = float("inf")
    initial_solutions = [route[1:-1] for route in initial_solutions]
    return initial_solutions

def fitness_function(individual_solution, distance_matrix):
    for route in individual_solution:
        if len(route) == 0:
            return 0
    obj_function_vector = objective_function_vector_bf(individual_solution,distance_matrix)
    return objective_function_minsum(obj_function_vector)

In [85]:
def biased_crossover(parent_a, parent_b, probability):
    child = []
    for gene_a, gene_b in zip(parent_a, parent_b):
        if random.random() < probability:
            child.append(gene_a)
        else:
            child.append(gene_b)
    return child

In [86]:
cities_distances = [[0,8,5,10,6,3,13,2,10,6,3,4,9,6,17],
         [8,0,9,6,12,7,4,9,15,9,10,6,5,9,11],
         [5,9,0,10,9,9,7,18,7,4,7,3,11,4,10],
         [10,6,10,0,18,2,10,6,5,4,9,12,8,8,10],
         [6,12,9,18,0,9,10,11,11,12,6,5,6,8,9],
         [3,7,9,2,9,0,6,8,9,10,5,2,7,10,6],
         [13,4,7,10,10,6,0,12,8,3,7,4,10,6,9],
         [2,9,18,6,11,8,12,0,7,3,6,18,17,9,2],
         [10,15,7,5,11,9,8,7,0,10,8,3,14,10,3],
         [6,9,4,4,12,10,3,3,10,0,12,5,9,6,3],
         [3,10,7,9,6,5,7,6,8,12,0,19,8,8,11],
         [4,6,3,12,5,2,4,18,3,5,19,0,12,2,9],
         [9,5,11,8,6,7,10,17,14,9,8,12,0,5,6],
         [6,9,4,8,8,10,6,9,10,6,8,2,5,0,2],
         [17,11,10,10,9,6,9,2,3,3,11,9,6,2,0]]

distance_matrix = cities_distances

number_of_salesmen = 3
num_of_genes = len(distance_matrix) - 1
initial_population_amount = 200
elite_fraction = 0.2
population_elite = round(elite_fraction*initial_population_amount)
population_mutants = 15
biased_crossover_rate = 0.7
number_of_generations = 3000
 
population = generate_population(initial_population_amount,num_of_genes)
population_decoded = [decoder_sequential_insertion(individual, number_of_salesmen, cities_distances) for individual in population]
print(population_decoded[0])
fitness_vector = [fitness_function(individual,cities_distances) for individual in population_decoded]
[decoder_sequential_insertion(item, 3, cities_distances) for item in population]

best_individual = population[0].copy()
best_solution = population_decoded[0].copy()
best_fit_function = fitness_vector[0]


[[14], [8], [3, 10, 7, 2, 13, 4, 6, 15, 9, 12, 5, 11]]


In [87]:
def BRKGA_MTSP(number_of_salesmen,num_of_genes, initial_population_amount, elite_fraction, population_mutants, biased_crossover_rate, number_of_generations, distance_matrix):
    max_iterations = 50
    best_iteration = 0
    population = generate_population(initial_population_amount,num_of_genes)
    population_decoded = [decoder_sequential_insertion(individual, number_of_salesmen, distance_matrix) for individual in population]
    fitness_vector = [fitness_function(individual, distance_matrix) for individual in population_decoded]

    best_individual = population[0].copy()
    best_solution = population_decoded[0].copy()
    best_fit_function = fitness_vector[0]

    population_elite = round(elite_fraction*initial_population_amount)

    for generation_number in range(number_of_generations):
        mutants = generate_population(population_mutants,num_of_genes)
        mutants_decoded = [decoder_sequential_insertion(mutant, number_of_salesmen, distance_matrix) for mutant in mutants]
        fitness_vector_mutants = [fitness_function(mutant, distance_matrix) for mutant in mutants_decoded]

        population += mutants
        fitness_vector += fitness_vector_mutants

        population_with_fit_function = []
        for individual, fit_function in zip(population, fitness_vector):
            population_with_fit_function.append([individual, fit_function])

        population_with_fit_function.sort(key = lambda x: x[1])

        '''
        for item in population_with_fit_function:
            print(item[1])
            '''
        '''
        alpha = 0.2
        fit_function_min = population_with_fit_function[0][1]
        fit_function_max = population_with_fit_function[-1][1]
        fit_function_gap = fit_function_max - fit_function_min
        fit_function_min, fit_function_max, fit_function_gap
        '''

        generation_best_individual = population_with_fit_function[0][0].copy()
        generation_best_solution = decoder_sequential_insertion(generation_best_individual, number_of_salesmen, distance_matrix)
        generation_best_fit_function = population_with_fit_function[0][1]

        if generation_best_fit_function < best_fit_function:
            best_individual = generation_best_individual.copy()
            best_solution = generation_best_solution.copy()
            best_fit_function = generation_best_fit_function
            print("Found new best in generation %d with fitness %d" % (generation_number, best_fit_function))
            best_iteration = generation_number
            print("--------New generation until halt: ",max_iterations,"------------------------------")
            print("")
            
            

        population_sorted = []

        for individual in population_with_fit_function:
            population_sorted.append(individual[0])

        elite_population = population_sorted[:population_elite]
        non_elite_population = population_sorted[population_elite:]

        population = []

        for new_individual in range(initial_population_amount):
            parent_a = random.choice(elite_population)
            parent_b = random.choice(non_elite_population)
            child = biased_crossover(parent_a, parent_b, biased_crossover_rate)
            population.append(child)

        if generation_number - best_iteration >= max_iterations:
            print("-----------TRAINING TERMINATED BEFORE FULL STOP AT GENERATION",generation_number,"--------------------")
            print("BEST INDIVIDUAL: ", best_individual)
            print("BEST SOLUTION: ")
            print_routes(best_solution)
            print("Objective Function Vector:", [calc_tour_length(route, distance_matrix) for route in best_solution])
            print("BEST FITNESS FUNCTION: ", best_fit_function)
            return

        population_decoded = [decoder_sequential_insertion(individual, number_of_salesmen, distance_matrix) for individual in population]
        fitness_vector = [fitness_function(individual,distance_matrix) for individual in population_decoded] 
    print("---------------------TRAINING TERMINATED AT GENERATION",generation_number,"----------------------------")
    print("BEST INDIVIDUAL: ", best_individual)
    print("BEST SOLUTION: ")
    print_routes(best_solution)
    print("Objective Function Vector:", [calc_tour_length(route, distance_matrix) for route in best_solution])
    print("BEST FITNESS FUNCTION: ", best_fit_function)

In [88]:
distance_matrix = cities_distances

number_of_salesmen = 3
num_of_genes = len(cities_distances)-1
initial_population_amount = 300
elite_fraction = 0.25
population_mutants = 30
biased_crossover_rate = 0.75
number_of_generations = 2000


BRKGA_MTSP(number_of_salesmen,num_of_genes, initial_population_amount, elite_fraction, population_mutants, biased_crossover_rate, number_of_generations,distance_matrix)


Found new best in generation 0 with fitness 63
--------New generation until halt:  50 ------------------------------

Found new best in generation 1 with fitness 60
--------New generation until halt:  51 ------------------------------

Found new best in generation 2 with fitness 59
--------New generation until halt:  53 ------------------------------

Found new best in generation 6 with fitness 58
--------New generation until halt:  59 ------------------------------

-----------TRAINING TERMINATED BEFORE FULL STOP AT GENERATION 59 --------------------
BEST INDIVIDUAL:  [0.512, 0.686, 0.186, 0.86, 0.21, 0.166, 0.13, 0.666, 0.712, 0.008, 0.944, 0.676, 0.98, 0.744]
BEST SOLUTION: 
[11]
[8]
[6, 4, 9, 15, 14, 12, 3, 10, 7, 2, 13, 5]
Objective Function Vector: [6, 4, 48]
BEST FITNESS FUNCTION:  58


In [89]:
'''

def generate_population_with_breakpoints(number_of_individuals, number_of_genes, number_of_salesmen):
    population = []
    for _ in range(number_of_individuals):
        individual = [(round(random.random(),3),__+2) for __ in range(number_of_genes)]
        individual = individual + [(round(random.random(),3),-1) for _ in range(number_of_salesmen-1)]
        population.append(individual)
    return population


def decoder_with_breakpoints(individual):
   
    individual.sort(key = lambda x: x[0]) # "LogLinear"
    ordered_indexes = [item[1] for item in individual] # Linear
    
    # Etapa de construção da solução
    number_of_salesmen = 1
    for gene in ordered_indexes:
        if gene == -1:
            number_of_salesmen += 1 
    solution = [[] for _ in range(number_of_salesmen)]
    solution_iterator = 0
    for gene in ordered_indexes:
        if gene != -1:
            solution[solution_iterator].append(gene)
        else:
            solution_iterator += 1
    return solution
'''

'\n\ndef generate_population_with_breakpoints(number_of_individuals, number_of_genes, number_of_salesmen):\n    population = []\n    for _ in range(number_of_individuals):\n        individual = [(round(random.random(),3),__+2) for __ in range(number_of_genes)]\n        individual = individual + [(round(random.random(),3),-1) for _ in range(number_of_salesmen-1)]\n        population.append(individual)\n    return population\n\n\ndef decoder_with_breakpoints(individual):\n   \n    individual.sort(key = lambda x: x[0]) # "LogLinear"\n    ordered_indexes = [item[1] for item in individual] # Linear\n    \n    # Etapa de construção da solução\n    number_of_salesmen = 1\n    for gene in ordered_indexes:\n        if gene == -1:\n            number_of_salesmen += 1 \n    solution = [[] for _ in range(number_of_salesmen)]\n    solution_iterator = 0\n    for gene in ordered_indexes:\n        if gene != -1:\n            solution[solution_iterator].append(gene)\n        else:\n            solution

In [90]:
import os

path = ['set I_MATRIX', 'set II_MATRIX']

for folder in path:
    csv_files = os.listdir(folder)
    for file in csv_files:
        filename = f'{folder}/{file}'
        print(filename)

set I_MATRIX/att532_10.txt
set I_MATRIX/att532_5.txt
set I_MATRIX/gtsp150_10.txt
set I_MATRIX/gtsp150_20.txt
set I_MATRIX/gtsp150_3.txt
set I_MATRIX/gtsp150_30.txt
set I_MATRIX/gtsp150_5.txt
set I_MATRIX/kroa200_10.txt
set I_MATRIX/kroa200_20.txt
set I_MATRIX/kroa200_3.txt
set I_MATRIX/kroa200_5.txt
set I_MATRIX/lin318_10.txt
set I_MATRIX/lin318_20.txt
set I_MATRIX/lin318_3.txt
set I_MATRIX/lin318_5.txt
set I_MATRIX/mtsp100_10.txt
set I_MATRIX/mtsp100_20.txt
set I_MATRIX/mtsp100_3.txt
set I_MATRIX/mtsp100_5.txt
set I_MATRIX/mtsp150_10.txt
set I_MATRIX/mtsp150_20.txt
set I_MATRIX/mtsp150_3.txt
set I_MATRIX/mtsp150_30.txt
set I_MATRIX/mtsp150_5.txt
set I_MATRIX/mtsp51_10.txt
set I_MATRIX/mtsp51_3.txt
set I_MATRIX/mtsp51_5.txt
set I_MATRIX/pcb1173_10.txt
set I_MATRIX/pcb1173_20.txt
set I_MATRIX/pcb1173_3.txt
set I_MATRIX/pcb1173_5.txt
set I_MATRIX/rand100_10.txt
set I_MATRIX/rand100_20.txt
set I_MATRIX/rand100_3.txt
set I_MATRIX/rand100_5.txt
set I_MATRIX/rat783_10.txt
set I_MATRIX/rat783

In [91]:
filename = 'set I_MATRIX/mtsp51_3.txt'

with open(filename) as f:
    lines = f.readlines()
    number_of_salesmen = int(filename.split('.')[0].split('_')[2])
number_of_salesmen

3

In [92]:
lines[0].split(" ")[-1]

'13.89\n'

In [93]:
distance_matrix = []

for line in lines:
    distance_matrix.append([float(item) for item in line.split(" ")])

num_of_genes = len(distance_matrix) - 1

In [94]:
import time
init_time = time.time()

BRKGA_MTSP(number_of_salesmen,
            num_of_genes,
            initial_population_amount, 
            elite_fraction, 
            population_mutants, 
            biased_crossover_rate, 
            number_of_generations,
            distance_matrix)

end_time = time.time()

print("Elapsed time:", end_time - init_time)

Found new best in generation 0 with fitness 457
--------New generation until halt:  50 ------------------------------

Found new best in generation 4 with fitness 456
--------New generation until halt:  54 ------------------------------

Found new best in generation 6 with fitness 452
--------New generation until halt:  60 ------------------------------

Found new best in generation 10 with fitness 452
--------New generation until halt:  70 ------------------------------

Found new best in generation 14 with fitness 451
--------New generation until halt:  84 ------------------------------

Found new best in generation 15 with fitness 450
--------New generation until halt:  99 ------------------------------

Found new best in generation 16 with fitness 449
--------New generation until halt:  115 ------------------------------

Found new best in generation 17 with fitness 448
--------New generation until halt:  132 ------------------------------

Found new best in generation 27 with fitn

In [96]:
solution = [[27],
[22],
[8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 19, 40, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32]]

a = [calc_tour_length(item,distance_matrix) for item in solution]
a, sum(a)

([16.12, 14.14, 415.95], 446.21)

### All time best: <br>

BEST SOLUTION:  <br>
[27] <br>
[22] <br>
[8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 19, 40, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32] <br>
Objective Function Vector: [16.12, 14.14, 415.95] <br>
BEST FITNESS FUNCTION:  446.21 <br>
Elapsed time: 61.13212871551514 <br>
 <br>